In [ ]:
from PIL import Image
import math

def gradual_stretch(
    input_path: str,
    output_path: str,
    x_start: int,
    width_multiplier: float = 2.0,
    affected_columns: int = 10,
    growth_factor: float = 1.5
):
    img = Image.open(input_path).convert("RGBA")
    width, height = img.size
    new_width = int(width * width_multiplier)

    if not (0 <= x_start < width):
        raise ValueError("x_start must be within image bounds")

    out = Image.new("RGBA", (new_width, height))

    # Copy unchanged left portion
    out.paste(img.crop((0, 0, x_start, height)), (0, 0))

    write_x = x_start
    remaining_width = new_width - x_start

    last_col = None

    # Generate stretch widths using exponential growth
    stretch_widths = [
        max(1, int(math.pow(growth_factor, i)))
        for i in range(affected_columns)
    ]

    for i, target_width in enumerate(stretch_widths):
        if remaining_width <= 0:
            break

        src_x = min(x_start + i, width - 1)
        col = img.crop((src_x, 0, src_x + 1, height))
        last_col = col

        w = min(target_width, remaining_width)
        stretched = col.resize((w, height), Image.NEAREST)
        out.paste(stretched, (write_x, 0))

        write_x += w
        remaining_width -= w

    # Fill all remaining space with final column
    if remaining_width > 0 and last_col is not None:
        stretched = last_col.resize((remaining_width, height), Image.NEAREST)
        out.paste(stretched, (write_x, 0))

    out.save(output_path)
    print(f"Saved stretched image to {output_path}")


gradual_stretch(
    input_path="input1.jpg",
    output_path="output1.2.png",
    x_start=960,
    width_multiplier=1,
    affected_columns=40,
    growth_factor=1.1
)

Saved stretched image to output1.2.png


In [25]:
from PIL import Image
import numpy as np
from tqdm.notebook import tqdm

def horizontal_gradient_blur(
    input_path,
    output_path,
    start_x,
    growth_multiplier
):
    img = Image.open(input_path).convert("RGBA")
    arr = np.array(img, dtype=np.float32)

    height, width, channels = arr.shape
    output = arr.copy()

    for y in tqdm(range(height), desc="Applying horizontal blur"):
        for x in range(start_x, width):
            radius = int((x - start_x) * growth_multiplier)

            if radius <= 0:
                continue

            left = max(0, x - radius)
            right = min(width - 1, x + radius)

            output[y, x] = arr[y, left:right + 1].mean(axis=0)

    result = Image.fromarray(np.clip(output, 0, 255).astype(np.uint8))
    result.save(output_path)

horizontal_gradient_blur(
    "output1.2.png",
    "outputoutput1.2.png",
    960 - 10,
    0.05
)

Applying horizontal blur:   0%|          | 0/2949 [00:00<?, ?it/s]

In [39]:
gradual_stretch(
    input_path="maxim-tolchinskiy-GA2y1XU9bIw-unsplash.jpg",
    output_path="output-maxim-tolchinskiy-GA2y1XU9bIw-unsplash.png",
    x_start=1165,
    width_multiplier=1,
    affected_columns=100,
    growth_factor=1.1
)

Saved stretched image to output-maxim-tolchinskiy-GA2y1XU9bIw-unsplash.png


In [41]:
horizontal_gradient_blur(
    "output-maxim-tolchinskiy-GA2y1XU9bIw-unsplash.png",
    "output-maxim-tolchinskiy-GA2y1XU9bIw-unsplash.png",
    1165 - 10,
    0.05
)

Applying horizontal blur:   0%|          | 0/2880 [00:00<?, ?it/s]